In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\0\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\0\test'
checkpoint_path = r'C:\Users\24306\Desktop\EfficientNetV2L\0\EfficientNetV2L.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[39.0, 4.0, 93.0, 45.0, 16.0, 47.0, 37.0, 96.0, 96.0, 37.0, 84.0, 40.0, 37.0, 93.0, 52.0, 16.0, 20.0, 95.0, 37.0, 20.0, 47.0, 37.0, 96.0, 80.0, 84.0, 100.0, 16.0, 37.0, 84.0, 93.0, 96.0, 96.0, 84.0, 100.0, 35.0, 80.0, 45.0, 95.0, 100.0, 37.0, 86.0, 7.0, 66.0, 47.0, 40.0, 100.0, 37.0, 96.0, 39.0, 80.0, 86.0, 30.0, 13.0, 37.0, 55.0, 32.0, 95.0, 32.0, 93.0, 84.0, 66.0, 84.0, 45.0, 23.0, 86.0, 23.0, 95.0, 37.0, 95.0, 13.0, 66.0, 93.0, 86.0, 55.0, 100.0, 10.0, 13.0, 100.0, 45.0, 10.0, 86.0, 86.0, 40.0, 66.0, 93.0, 47.0, 32.0, 55.0, 37.0, 84.0, 13.0, 84.0, 40.0, 7.0, 66.0, 80.0, 84.0, 35.0, 95.0, 37.0, 7.0, 96.0, 84.0, 66.0, 16.0, 16.0, 66.0, 55.0, 37.0, 40.0, 10.0, 96.0, 95.0, 96.0, 84.0, 16.0, 10.0, 95.0, 100.0, 45.0, 45.0, 80.0, 16.0, 4.0, 7.0, 37.0, 39.0, 84.0, 47.0, 4.0, 86.0, 40.0, 66.0, 55.0, 30.0, 100.0, 96.0, 30.0, 52.0, 23.0, 52.0, 100.0, 100.0, 95.0, 35.0, 55.0, 32.0, 10.0, 66.0, 80.0, 96.0, 13.0, 93.0, 32.0, 66.0, 13.0, 13.0, 84.0, 80.0, 96.0, 80.0, 7.0, 37.0, 80.0, 16.0, 96.0, 3

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
EfficientNetV2L = tf.keras.applications.efficientnet_v2.EfficientNetV2L(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
EfficientNetV2L.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = EfficientNetV2L(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-l (Functiona  (None, 7, 7, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1281      
                                                                 
Total params: 117,748,129
Trainable params: 117,235,553
Non-trainable params: 512,576
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('EfficientNetV2L_loss_0.txt',history.history['loss'])
#要改
np.savetxt('EfficientNetV2L_valloss_0.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Fri Apr 28 10:50:35 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 1068.5779

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\0\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\0\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 639s 1s/step - loss: 1068.5779 - val_loss: 728.5102
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 331.5332

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\0\EfficientNetV2L.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\0\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 1202s 2s/step - loss: 0.4826 - val_loss: 0.0880
Epoch 247/300
591/591 [==============================] - 1049s 2s/step - loss: 0.3539 - val_loss: 0.1482
Epoch 248/300
591/591 [==============================] - 1042s 2s/step - loss: 0.2703 - val_loss: 0.0888
Epoch 249/300
591/591 [==============================] - 1021s 2s/step - loss: 0.4954 - val_loss: 0.1349
Epoch 250/300
591/591 [==============================] - 1030s 2s/step - loss: 0.4010 - val_loss: 0.1142
Epoch 251/300
591/591 [==============================] - 1025s 2s/step - loss: 0.3233 - val_loss: 0.1587
Epoch 252/300
591/591 [==============================] - 1021s 2s/step - loss: 0.2951 - val_loss: 1.0152
Epoch 253/300
591/591 [==============================] - 1013s 2s/step - loss: 0.2864 - val_loss: 0.2802
Epoch 254/300
591/591 [==============================] - 1012s 2s/step - loss: 0.3529 - val_loss: 0.1385
Epoch 255/300
591/591 [==============================] - 1011s 2s/ste

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\0\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\0\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 1295s 2s/step - loss: 0.3195 - val_loss: 0.0858
Epoch 278/300
591/591 [==============================] - 998s 2s/step - loss: 0.3748 - val_loss: 0.1005
Epoch 279/300
591/591 [==============================] - 990s 2s/step - loss: 0.3516 - val_loss: 0.1136
Epoch 280/300
591/591 [==============================] - 995s 2s/step - loss: 0.2485 - val_loss: 2.0653
Epoch 281/300
591/591 [==============================] - 993s 2s/step - loss: 0.3975 - val_loss: 0.2960
Epoch 282/300
591/591 [==============================] - 993s 2s/step - loss: 0.2332 - val_loss: 0.2914
Epoch 283/300
591/591 [==============================] - 993s 2s/step - loss: 0.2157 - val_loss: 0.1018
Epoch 284/300
591/591 [==============================] - 995s 2s/step - loss: 0.3265 - val_loss: 1.2500
Epoch 285/300
591/591 [==============================] - 990s 2s/step - loss: 0.3236 - val_loss: 0.2212
Epoch 286/300
591/591 [==============================] - 991s 2s/step - loss: